## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from matplotlib import colors

# TODO: Fill this in based on where you saved the training and testing data

training_file = "./train.p"
validation_file= "./valid.p"
testing_file = "./test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

signnames = pd.read_csv("./signnames.csv")

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = len (X_train)

# TODO: Number of validation examples
n_validation = len (X_valid)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

n_valid = len(X_valid)

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

def plot_histograms(n_classes):
    """Ploting distributions of each set"""
    def count_labels(y, n_cls=n_classes):
        temp = y.tolist()
        labels_count = np.zeros(n_classes)
        for item in range(n_classes): 
            labels_count[item] = temp.count(item)
        return labels_count
    
    x_axis = list(np.linspace(0, n_classes-1, num=n_classes))
    plt.figure(num=1)
    n_tr_lbls = count_labels(y_train)
    plt.bar(x_axis, n_tr_lbls)
    plt.title('Distribution of classes in X_train (n=%d)'%len(X_train))
    plt.xlabel('classes: 0 - %d'%(n_classes - 1))
    plt.ylabel('number of images')

    plt.figure(num=2)
    n_vl_lbls = count_labels(y_valid)
    plt.bar(x_axis, n_vl_lbls)
    plt.title('Distribution of classes in X_valid (n=%d)'%len(X_valid))
    plt.xlabel('classes: 0 - %d'%(n_classes - 1))
    plt.ylabel('number of images')
    
    plt.figure(num=3)
    n_tt_lbls = count_labels(y_test)
    plt.bar(x_axis, n_tt_lbls)    
    plt.title('Distribution of classes in X_test (n=%d)'%len(X_test))
    plt.xlabel('classes: 0 - %d'%(n_classes - 1))
    plt.ylabel('number of images')

plot_histograms(n_classes)

### Data visualization

In [ ]:
from ipywidgets import widgets
import traitlets
from IPython.display import display
import matplotlib.gridspec as gridspec

x_train_grped = {}
for label in np.unique(y_train).tolist():
    x_train_grped[label] = []
for index in range (n_train):
    x_train_grped[y_train[index]].append(index)

label_slider = widgets.IntSlider(max=n_classes, description="Label")
label_slider.max = n_classes - 1
index_slider = widgets.IntSlider(description="Image index")
description = widgets.Text(disabled=True)
    
def on_label_value_changed(change):
    index_slider.max = len(x_train_grped[label_slider.value]) - 1
index_slider.observe(on_label_value_changed, names='value')

img_index = 0
dummy_res = None
def display_images(index, label):
    global img_index, dummy_res
    index_slider.max = len(x_train_grped[label_slider.value]) - 1
    plt.figure(figsize=(3, 3))
    data = x_train_grped[label_slider.value]
    img_index = data[index_slider.value] 
    #description.value = "Image data at X_train [%d]"%img_index
    
    plt.imshow(X_train[img_index])
    plt.title(signnames['SignName'][label_slider.value])
    
caption = widgets.Label(value='Please select a label and a corresponding index \
                                        in the container of grouped training data')
display(caption)
# redirect output to variable and kwargs __output_result to avoid the verbose printing
my_widget = widgets.interact(display_images, label=label_slider, index=index_slider, \
                             __output_result=False)
#my_widget = widgets.interact(display_images, label=label_slider, index=index_slider, \
#                             desc=description, __output_result=False)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
from skimage.transform import warp, warp_coords
from scipy.ndimage.interpolation import map_coordinates
from skimage.transform import ProjectiveTransform
from skimage import color
from skimage import exposure, img_as_float
import random
import collections
import os.path

rotation_range = np.linspace(.15, .22, num=10, dtype=float)
translation_range = np.linspace(-5, 5, num=10, dtype=float)
scaling_range = np.linspace(.9, 1.5, num=15, dtype=float)
rotation_range = rotation_range.tolist()
translation_range = translation_range.tolist()
scaling_range = scaling_range.tolist()
AUG_TRAIN_FILE = "./train_a.p" 
X_train_a = None
y_train_a = None

def rnd_transform(orig_img, transform):
    result = None
    t_matrix = None
    img = np.copy(orig_img)
    
    if transform == "rotation":
        index = random.randint(0, len(rotation_range)-1)
        angle = rotation_range[index]
        
## Matrix represented as [ [x_scale,  x_skew,  t_x]
##                         [y_scale,  y_skew,  t_y]
##                         [x_position, y_position, 1]]
##                
        sin_val = np.array([np.sin(angle), -np.sin(angle)])
        random.shuffle(sin_val)
        
        t_matrix = np.array([[np.cos(angle), sin_val[0], 0],\
                             [sin_val[1], np.cos(angle), 0],\
                             [0, 0, 1]])
    elif transform == "translation":
        x_index = random.randint(0, len(translation_range)-1)
        x_tr = translation_range[x_index]
        y_index = random.randint(0, len(translation_range)-1)        
        y_tr = translation_range[y_index]
        #print(x_tr, y_tr)
        t_matrix = np.array([[1, 0, x_tr],\
                             [0, 1, -y_tr],\
                             [0, 0, 1]])
    elif transform == "scaling":
        x_index = random.randint(0, len(scaling_range)-1)
        x_sc = scaling_range[x_index]
        #print (x_sc)
        y_index = random.randint(0, len(scaling_range)-1)        
        y_sc = scaling_range[y_index]
        #print(y_sc)
        t_matrix = np.array([[x_sc, 0, 0],\
                              [0, y_sc, 0],\
                              [0, 0, 1]])
    else:
        pass
    if t_matrix is not None:
        t_trans = ProjectiveTransform(matrix=t_matrix)
        img = warp(img, t_trans, mode='edge', clip=True,\
                   order=1, output_shape=(img.shape[0], img.shape[1]))  
    return img


def rescale_intensity(img):
    p2, p98 = np.percentile(img, (2, 98))
    img_ = exposure.rescale_intensity(img, in_range=(p2, p98))
    return img_
    
def augment_data(threshold=750, save_as=True):
    #find classes with less than "threshold" amount of data
    tr_counter=collections.Counter(y_train.tolist())
    tr_counter = collections.OrderedDict(tr_counter)
    threshold = 750
    below_treshold_cls = [key for key in list(tr_counter.keys())\
                                      if tr_counter[key] < threshold]
    trf_op = ['rotation', 'translation', 'scaling']
    
    X_train_a = np.copy(X_train)
    y_train_a = np.copy(y_train)
    print (len(X_train_a))
    print (len(y_train_a))
    for cls in below_treshold_cls:
        print ("Processing label %d ..."%cls)
        cls_indexes = np.where(y_train==cls)[0]
        cls_indexes = cls_indexes.tolist()
        for index in cls_indexes:
            tmp_orig = np.copy(X_train[index])
            for op in trf_op:
                tmp = rnd_transform(tmp_orig, transform=op)
                # update X_train and y_train
                X_train_a = np.append(X_train_a, np.array([tmp]), axis=0)
                y_train_a = np.append(y_train_a, cls)
    # Save data into new file
    outfile = open(AUG_TRAIN_FILE, 'wb')
    train_aug = {'features': X_train_a, 'labels': y_train_a}
    pickle.dump(train_aug, outfile)
    outfile.close()
    
    # Rename new data
    X_train = X_train_a
    y_train = y_train_a
    print("Data augmentation finished.")




### Data Augmentation & Pre-processing

#### Fake data generation (Caution: takes a long time!)

In [ ]:
AUGMENT_DATA = True
# Load or generate fake data
if os.path.isfile(AUG_TRAIN_FILE) and\
            os.access(AUG_TRAIN_FILE, os.R_OK) and AUGMENT_DATA:
    with open(AUG_TRAIN_FILE, mode='rb') as f:
        train_a = pickle.load(f)
        X_train_a, y_train_a = train_a['features'], train_a['labels']
        X_train = X_train_a
        y_train = y_train_a
        X_train, y_train = shuffle(X_train, y_train)
elif AUGMENT_DATA:
    augment_data()
    X_train, y_train = shuffle(X_train, y_train)
else:
    print("No data augmentation applied!")

# Histograms
plot_histograms(n_classes)


#### Normalization, grayscale

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
GRAY_MODE = False

if GRAY_MODE:
    X_train = np.dot(X_train[...,:3], [0.299, 0.587, 0.114])        
    X_valid = np.dot(X_valid[...,:3], [0.299, 0.587, 0.114])
    # add last channel
    X_train = X_train.reshape(X_train.shape + (1,))
    X_valid = X_valid.reshape(X_valid.shape + (1,))

X_train = X_train.astype(np.float32)/255.0
X_valid = X_valid.astype(np.float32)/255.0
X_test  = X_test.astype(np.float32)/255.0

X_train, y_train = shuffle(X_train, y_train)

# Rescale intensity of training examples
map(rescale_intensity, X_train)

# Rescale intensity of validation examples
map(rescale_intensity, X_valid)

# Rescale intensity of test examples
map(rescale_intensity, X_test)


### Model Architecture

#### Using Keras with TensorFlow backend

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.utils.np_utils import to_categorical
import keras.backend as K
from keras import optimizers
from keras import initializers
from keras import regularizers
import tensorflow as tf
from keras.models import load_model
from keras.callbacks import EarlyStopping

KERAS_MODE = True

#Converts a class vector (integers) to binary class matrix.
#E.g. for use with categorical_crossentropy.
y_train_cat = to_categorical(y_train)
y_valid_cat = to_categorical(y_valid)
y_test_cat = to_categorical(y_test)

BATCH_SIZE_KE = 128
EPOCHS_KE = 500 #15 #100
learning_rate_ke = 0.001


def LeNet_Ke():
    K.clear_session()
    model = Sequential()
    mu = 0
    sigma = 0.1
    
    # Conv2D
    model.add(Conv2D(6, (5, 5), activation='relu',\
                     input_shape=(32, 32, 3), padding='valid',\
                     kernel_initializer=initializers.TruncatedNormal(stddev=sigma),\
                     #kernel_regularizer='l2',\
                     #kernel_regularizer=regularizers.l2(0.01),\
                     #activity_regularizer=regularizers.l1(0.01),
                     use_bias=True,
                     bias_initializer=initializers.Zeros(),
                     name='conv2d_1'))
    # MaxPool2D
    model.add(MaxPool2D(pool_size=(2, 2), name='maxpool2d_1'))
    
    # Conv2D   
    model.add(Conv2D(16, (5, 5), activation='relu', padding='valid',\
                    kernel_initializer=initializers.TruncatedNormal(stddev=sigma),\
                    #kernel_regularizer='l2',\
                    #kernel_regularizer=regularizers.l2(0.01),
                    #activity_regularizer=regularizers.l1(0.01), 
                    use_bias=True,
                    bias_initializer=initializers.Zeros(),
                    name='conv2d_2'))
    # MaxPool2D
    model.add(MaxPool2D(pool_size=(2, 2), name='maxpool2d_2'))
    
    # Dropout
    model.add(Dropout(0.25, name='dpo_1'))
    model.add(Flatten(name='fln_1'))
    
    # Fully Connected
    model.add(Dense(120, activation='relu',\
                    kernel_initializer=initializers.TruncatedNormal(stddev=sigma),\
                    #kernel_regularizer='l2',\
                    #kernel_regularizer=regularizers.l2(0.01),\
                    #activity_regularizer=regularizers.l1(0.01), 
                    use_bias=True,
                    bias_initializer=initializers.Zeros(),
                    name='fc_1'))

    # Fully Connected
    model.add(Dense(84, activation='relu', \
                    kernel_initializer=initializers.TruncatedNormal(stddev=sigma),\
                    #kernel_regularizer='l2',\
                    #kernel_regularizer=regularizers.l2(0.01),\
                    #activity_regularizer=regularizers.l1(0.01),                     
                    use_bias=True,
                    bias_initializer=initializers.Zeros(),
                    name='fc_2'))  

    model.add(Dropout(0.5, name='dpo_2'))
    
    # Fully Connected
    model.add(Dense(n_classes,\
                    kernel_initializer=initializers.TruncatedNormal(stddev=sigma),\
                    #kernel_regularizer='l2',\
                    #kernel_regularizer=regularizers.l2(0.01),\
                    #activity_regularizer=regularizers.l1(0.01), 
                    use_bias=True, 
                    bias_initializer=initializers.Zeros(),
                    activation='softmax',
                    name='fc_3'))
    
    return model 
    

if KERAS_MODE:
    with tf.device('/gpu:0'):
        model = LeNet_Ke()
        model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=learning_rate_ke),
              metrics=['accuracy'])

        # Model Summary
        model.summary()
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=2)
        hist = model.fit(X_train, y_train_cat, batch_size=BATCH_SIZE_KE,\
              validation_data=(X_valid, y_valid_cat),\
              epochs=EPOCHS_KE, verbose=1, shuffle=True, callbacks=[early_stopping])
        
        # Save history
        tr_outfile = open('./training_history', 'wb')
        pickle.dump(hist.history, tr_outfile)
        print("Training history saved.")
        
        ## TODO
        ## plot accuracy, loss, etc
        print()
        print()
        # Print history 
        print("Model history")
        print(hist.history)
        # Save the model
        model.save('./lenet_ke.h5')


#### Model (Keras) prediction on test samples

In [ ]:
model = load_model('./lenet_ke.h5')

y_test_pred = model.predict_classes(X_test[0:5])
print (y_test_pred)

# TODO
# show test accuracy, plots, etc


### Model Architecture

#### Using TensorFlow
The `EPOCH` and `BATCH_SIZE` values affect the training speed and model accuracy.


In [ ]:
import tensorflow as tf

rate = 0.001
EPOCHS = 1000 # 20 # 100 
BATCH_SIZE = 128 # test of 256 


#### Output

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, X_train.shape[3], 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    #conv1_b = tf.Variable(tf.random_normal([6]))
    #conv1_b = tf.Variable(tf.constant(0.05, shape=[6]))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    #conv2_b = tf.Variable(tf.constant(0.05, shape=[16]))
    
    #conv2_b = tf.Variable(tf.random_normal([16]))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)
    
    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.constant(0.05, shape=[120]))
    fc1_b = tf.Variable(tf.zeros(120))
    #fc1_b = tf.Variable(tf.random_normal([120]))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)
    # Dropout
    #fc1    = tf.nn.dropout(fc1, 0.25)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.constant(0.05, shape=[84]))
    fc2_b  = tf.Variable(tf.zeros(84))
    #fc2_b  = tf.Variable(tf.random_normal([84]))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)
    # Dropout
    #fc2    = tf.nn.dropout(fc2, 0.5)
    
    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = n_classes.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, n_classes), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(n_classes)) 
    #fc3_b = tf.Variable(tf.constant(0.05, shape=[n_classes]))
    #fc3_b = tf.Variable(tf.random_normal([n_classes]))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits


### Train, Validate and Test the Model

#### Features and Labels

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, X_train.shape[3]))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

logits = LeNet(x)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

#### Model Evaluation

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        loss, accuracy = sess.run([loss_operation, accuracy_operation], feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return loss, total_accuracy / num_examples

#### Model Trainning

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        
        X_valid, y_valid = shuffle(X_valid, y_valid)
        loss, validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Loss = {:.3f}".format(loss))
        print()
        
    #saver.save(sess, './lenet')
    print("Model saved")

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")